In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
df.isnull().sum()
df = df.dropna(subset=['question1', 'question2'])
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [5]:
new_df = df.sample(n=40000,random_state=2)

In [6]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
331535,How can I learn Norwegian?,What is the quickest way to learn Norwegian?
45407,How are currency rates determined?,Where and how are exchange rates determined?
286200,What is substitution?,What is a substitute for caciocavallo?
157195,How can I make iPhone 4s faster with IOS 9.2?,I have an iPhone 4S. How do I make it faster a...
154346,How can I help my girlfriend cope with her par...,What can I do to help my girlfriend through he...


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=4000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(40000, 8000)

In [8]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
331535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
286200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
temp_df['is_duplicate'] = new_df['is_duplicate']
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,is_duplicate
331535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
45407,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
286200,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
157195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
154346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=2)

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.755

In [12]:
import pickle

pickle.dump(rf,open('model_base.pkl','wb'))
pickle.dump(cv,open('cv_base.pkl','wb'))

In [19]:
def predict_similarity(q1, q2, vectorizer, model):
    
    input_pair = [q1, q2]
    vectorized = vectorizer.transform(input_pair).toarray()  # Shape: (2, 4000)
    q1_vec, q2_vec = np.vsplit(vectorized, 2)  # Each shape: (1, 4000)
    combined = np.concatenate((q1_vec, q2_vec), axis=1)  # Shape: (1, 8000)
    prob = model.predict_proba(combined)[0][1]    
    return prob

In [37]:
predict_similarity('How can I learn Norwegian in the quickest way?','What is the quickest way to learn Norwegian?',cv,rf)

0.41